# Using BLAST from Biopython

In [ ]:
!pip install biopython

# import Biopython functions
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import Entrez

## Part One - Retreiving Sequences

In [ ]:
# load the DNA Sequence for RBP4

gene_id = 'NM_006744.3' # human RBP4

Entrez.email = 'A.N.Other@example.com'

# read the first sequence
handle = Entrez.efetch(db="nucleotide", id=gene_id, rettype="gb", retmode="text")
gene = SeqIO.read(handle, "genbank")
handle.close()

print(gene.description)

In [ ]:
# this is the whole transcript
# location of the coding sequence is here
for f in gene.features:
    if f.type=='CDS':
        print('Coding sequence at:',f.location)
        cds_loc = f.location


In [ ]:
#knowing this work out where the 3'UTR sequence must be

seq3utr = gene.seq[cds_loc.end:]

print(seq3utr)


## Part Two Runing BLAST and Parsing Output

In [ ]:
# run a BLAST using only the 3'UTR seuence

# there are several choices for the database to query:
# nt - the full database
# refseq_rna - Curated (NM_, NR_) plus predicted (XM_, XR_) sequences from NCBI Reference Sequence Project
# refseq_genomic - Genomic sequences from NCBI Reference Sequence Project
# for available databases, see:
# ftp://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf

database = 'refseq_rna'
result_handle = NCBIWWW.qblast("blastn", database, seq3utr)


NB the search above is simple, you can pass all the parameters to the search by modifying the qblast query string, for example:-

> blast_handle = NCBIWWW.qblast("blastp", "nr",
                              peptide_seq,
                              expect=200000.0,
                              filter=False,
                              word_size=2,
                              composition_based_statistics=False,
                              matrix_name="PAM30",
                              gapcosts="9 1",
                              hitlist_size=1000)

In [ ]:
# Parse the retuned structure
blast_records = NCBIXML.parse(result_handle)

In [ ]:
# take the first record (we only did one search, so there is only one)
item = next(blast_records)

In [ ]:
E_VALUE_THRESH = 1#0.05

for alignment in item.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print('****************************')
            print('sequence:', alignment.title)
            print('length:', alignment.length)
            print('e value:', hsp.expect)
            print(hsp.query[0:75] + '...')
            print(hsp.match[0:75] + '...')
            print(hsp.sbjct[0:75] + '...')